<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 00 - Scenario Overview and System Setup

In this set of tutorials you'll work with an end-to-end scenario that uses SQL Server 2019's big data clusters to solve real-world problems. 


## Wide World Importers

Wide World Importeres (WWI) is a traditional brick and mortar business with a long track record of success, generating profits through strong retail store sales of their unique offering of affordable products from around the world. 

WWI has a typical N-Tier application that provides a series of terminals, a Business Logic layer, and a Database back-end. WWI corporate has now added a new web and a mobile commerce application to their platform, but not through the IT Department. This  has generated a significant amount of additional data and data formats. These new platforms were added without integrating into the OLTP system data or Business Intelligence infrastructures. As a result, "silos" of data stores have developed.

Now they want to expand their reach to customers around the world through web and mobile e-commerce. But they don't want to just simply make their inventory available online. They want to build upon their track record of strong customer connections, and engage with their customers through personalized, high-quality application experiences that incorporate data and intelligence.
The technology team at WWI has recognized that moving to an omni-channel strategy has quickly outgrown their ability to handle data. They anticipate the following solutions needed to reach more customers and grow the business:

- Scale data systems to reach more consumers
- Unlock business insights from multiple sources of structured and unstructured data
- Apply deep analytics with high-performance responses
- Enable Artificial Intelligence into apps to actively engage with customers

In this series of tutorials, you will build a solution using the scale-out features of SQL Server 2019, Data Virtualization, Data Marts, and the Data Lake features. 

## Running these Tutorials

- You can read through the output of these completed tutorials if you wish - or:

- You can follow along with the steps you see in these tutorials by copying the code into a SQL Query window and Spark Notebook using the Azure Data Studio tool, or you can click here to download these Jupyter Notebooks and run them in Azure Data Studio for a hands-on experience.
 
- If you would like to run the tutorials, you'll need a SQL Server 2019 big data cluster and the client tools installed. If you want to set up your own cluster, <a href="https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-get-started?view=sqlallproducts-allversions" target="_blank">click this reference and follow the steps you see there for the server and tools you need</a>.

- You will need to have the following: 
    - Your **Knox Password**
    - The **Knox IP Address**
    - The `sa` **Username** and **Password** to your Master Instance
    - The **IP address** to the SQL Server big data cluster Master Instance 
    - The **name** of your big data cluster

For a complete workshop on SQL Server 2019's big data clusters, <a href="https://github.com/Microsoft/sqlworkshops/tree/master/sqlserver2019bigdataclusters" target="_blank">check out this resource</a>.

## Copy Database backups to the SQL Server 2019 big data cluster Master Instance

The first step for the solution is to copy the database backups from WWI from their location on the cloud and then up to your cluster. 

These commands use the `curl`program to pull the files down. [You can read more about curl here](https://curl.haxx.se/). 

The next set of commands use the `kubectl` command to copy the files from where you downloaded them to the data directory of the SQL Server 2019 bdc Master Instance. [You can read more about kubectl here](https://kubernetes.io/docs/reference/kubectl/overview/). Note that you will need to replace the section of the script marked with `ReplaceWithClusterName` with the name of your SQL Server 2019 bdc. (It does not need single or double quotes, just the letters of your cluster name.)

In the next tutorial you will restore these databases on the Master Instance.

In [2]:
!md d:\temp

!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/WWI.bak" -o d:\temp\WWI.bak
!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/WWIDW.bak" -o d:\temp\WWIDW.bak
!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/WWIDW.bak" -o d:\temp\NYC.bak


!kubectl cp d:/temp/WWI.bak mssql-master-pool-0:/var/opt/mssql/data -c mssql-server -n <ReplaceWithClusterName>
!kubectl cp d:/temp/WWIDW.bak mssql-master-pool-0:/var/opt/mssql/data -c mssql-server -n <ReplaceWithClusterName>
!kubectl cp d:/temp/NYC.bak mssql-master-pool-0:/var/opt/mssql/data -c mssql-server -n <ReplaceWithClusterName>

## Copy Exported Data to Storage Pool

**TODO:** Explain curls to hdfs

In [ ]:
!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/product_review_data?op=MKDIRS"
!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/adventureworks_export?op=MKDIRS"

!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/product_reviews.csv" -o d:\temp\product_reviews.csv
!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/customers.csv" -o d:\temp\customers.csv
!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/web_clickstreams.csv" -o d:\temp\web_clickstreams.csv
!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/stockitemholdings.csv" -o d:\temp\stockitemholdings.csv

!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/product_review_data/product_reviews.csv?op=create&overwrite=true" -H "Content-Type: application/octet-stream" -T "product_reviews.csv"
!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/adventureworks_export/customers.csv?op=create&overwrite=true" -H "Content-Type: application/octet-stream" -T "customers.csv.csv"
!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/adventureworks_export/web_clickstreams.csv?op=create&overwrite=true" -H "Content-Type: application/octet-stream" -T "web_clickstreams.csv"
!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/adventureworks_export/stockitemholdings.csv?op=create&overwrite=true" -H "Content-Type: application/octet-stream" -T "stockitemholdings.csv"

## Next Step: Working with the SQL Server 2019 big data cluster Master Instance

**TODO:** Add Link